In [1]:
import logging

logging.basicConfig(level=logging.INFO, filename='log/app.log', filemode='w', format='%(name)s - %(levelname)s - %(asctime)s: %(message)s')
logging.info('start')

In [2]:
import numpy as np
import matplotlib.cm as cm
import pandas as pd

from possibilearn import *
from possibilearn.kernel import GaussianKernel

from sklearn.model_selection import KFold

source = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'

iris_df = pd.read_csv(
    filepath_or_buffer=source,
    header=None,
    sep=',')

iris_df.columns=['sepal_len', 'sepal_wid', 'petal_len', 'petal_wid', 'class']
iris_df.dropna(how="all", inplace=True) # drops the empty line at file-end

iris_df.tail()

,sepal_len,sepal_wid,petal_len,petal_wid,class
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica
149,5.9,3.0,5.1,1.8,Iris-virginica


In [3]:
iris_values = iris_df.iloc[:,0:4].values
iris_labels = iris_df.iloc[:,4].values

classes = ('Iris-virginica', 'Iris-versicolor', 'Iris-setosa')

In [4]:
from sklearn.preprocessing import StandardScaler

iris_values_std = StandardScaler().fit_transform(iris_values)

In [5]:
cs = [0.1, 10, 100]
ks = [.5, .225, .1]
message = "cs: "
for i in range(len(cs)):
    message += '%s, ' % cs[i]

logging.info(message)
message = "ks: "
for i in range(len(ks)):
    message += '%s, ' % ks[i]

print(message)
logging.info('cs: %s', message)
#logging.info('ks: %s', ks)

ks: 0.5, 0.225, 0.1, 


In [6]:
def to_membership_values(labels, target):
    return [1 if l==target else 0 for l in labels]

def classify(item, est_mu, classes):
    return sorted([(l, est_mu[l](item)) for l in classes], key=lambda i:i[1], reverse=True)[0][0]

def get_generator(d):
    return lambda m: (-4 + np.random.random(d*m) * 8).reshape((m, d))
    return (-4 + np.random.random(2*m) * 8).reshape((m, 2))

In [7]:
mu = {}
mu_train = {}
mu_test = {}

est_mu = {}

In [9]:
from sklearn.decomposition import PCA

num_holdouts = 4

for n in range(2, 5):
    iris_values_tr = PCA(n_components=n).fit_transform(iris_values_std)
    results_df = pd.DataFrame()
    
    for c in cs:
        for k in ks:
            for train, test in KFold(n_splits=num_holdouts, shuffle=True, random_state=3).split(iris_values_tr):
                for target in classes:
                    mu[target] = to_membership_values(iris_labels, target)
                    mu_train[target] = [mu[target][i] for i in train]
                    mu_test[target] = [mu[target][i] for i in test]

                train_values = [iris_values_tr[i] for i in train]
                test_values = [iris_values_tr[i] for i in test]

                train_labels = [iris_labels[i] for i in train]
                test_labels = [iris_labels[i] for i in test]

                for target in classes:
                    est_mu[target], _ = possibility_learn(train_values,
                                                          mu_train[target],
                                                          c=c,
                                                          k=GaussianKernel(k),
                                                          sample_generator=get_generator(n))
                print(c, k)
                
                results_train = list(zip(range(len(train_values)), map(lambda item: classify(item, est_mu, classes), train_values), train_labels))
                results_test = list(zip(range(len(test_values)), map(lambda item: classify(item, est_mu, classes), test_values), test_labels))

                perf_train = len([r for r in results_train if r[1] == r[2]])/len(train_values)
                perf_test = len([r for r in results_test if r[1] == r[2]])/len(test_values)
                
                logging.info("performance test: %s", perf_test)
                logging.info("performance train: %s", perf_train)
                logging.info("C: %s", c)
                logging.info("K: %s", k)

                results_df = results_df.append({'train': perf_train,
                                                'test': perf_test,
                                                 'C': c,
                                                 'K': k
                                               },
                                  ignore_index=True)
            print("Components: ", n)
            display(results_df)

Academic license - for non-commercial use only
0.1 0.5
0.1 0.5
0.1 0.5
0.1 0.5
Components:  2


,C,K,test,train
0,0.1,0.5,0.894737,0.937500
1,0.1,0.5,0.894737,0.928571
2,0.1,0.5,0.945946,0.929204
3,0.1,0.5,0.891892,0.938053


0.1 0.225
0.1 0.225
0.1 0.225
0.1 0.225
Components:  2


,C,K,test,train
0,0.1,0.500,0.894737,0.937500
1,0.1,0.500,0.894737,0.928571
2,0.1,0.500,0.945946,0.929204
3,0.1,0.500,0.891892,0.938053
4,0.1,0.225,0.842105,0.991071
5,0.1,0.225,0.894737,0.991071
6,0.1,0.225,0.945946,0.982301
7,0.1,0.225,0.864865,0.955752


0.1 0.1
0.1 0.1
0.1 0.1
0.1 0.1
Components:  2


...

,C,K,test,train
0,0.1,0.500,0.921053,1.00000
1,0.1,0.500,0.947368,1.00000
2,0.1,0.500,0.972973,1.00000
3,0.1,0.500,0.945946,0.99115
4,0.1,0.225,0.894737,1.00000
5,0.1,0.225,0.921053,1.00000
6,0.1,0.225,0.972973,1.00000
7,0.1,0.225,0.945946,1.00000
8,0.1,0.100,0.894737,1.00000
9,0.1,0.100,0.921053,1.00000


100 0.225
100 0.225
100 0.225
100 0.225
Components:  4


,C,K,test,train
0,0.1,0.500,0.921053,1.00000
1,0.1,0.500,0.947368,1.00000
2,0.1,0.500,0.972973,1.00000
3,0.1,0.500,0.945946,0.99115
4,0.1,0.225,0.894737,1.00000
5,0.1,0.225,0.921053,1.00000
6,0.1,0.225,0.972973,1.00000
7,0.1,0.225,0.945946,1.00000
8,0.1,0.100,0.894737,1.00000
9,0.1,0.100,0.921053,1.00000


100 0.1
100 0.1
100 0.1
100 0.1
Components:  4


,C,K,test,train
0,0.1,0.500,0.921053,1.00000
1,0.1,0.500,0.947368,1.00000
2,0.1,0.500,0.972973,1.00000
3,0.1,0.500,0.945946,0.99115
4,0.1,0.225,0.894737,1.00000
5,0.1,0.225,0.921053,1.00000
6,0.1,0.225,0.972973,1.00000
7,0.1,0.225,0.945946,1.00000
8,0.1,0.100,0.894737,1.00000
9,0.1,0.100,0.921053,1.00000
